In [ ]:
import pandas as pd

## 1 Load Data

In [ ]:
file_path = r"path\to\data.xlsx"
sheets = ['客户损益', '收入明细', '成本明细集成']
df_dict = pd.read_excel(file_path, sheet_name=sheets)

output_file = "output.xlsx"

In [ ]:
warehouse_mapping = {
    "CA1": "美国洛杉矶1号仓",
    "CA2": "美国洛杉矶2号仓",
    "CA3": "美国洛杉矶3号仓",
    "GA1": "美国亚特兰大1号仓",
    "NJ1": "美国新泽西1号仓",
    "NJ2": "美国新泽西2号仓",
    "NJ3": "美国新泽西3号仓"
}

customer_profit = df_dict['客户损益']
warehouse_name = customer_profit['仓'].map(warehouse_mapping)
customer_profit.drop(columns=['仓'], inplace=True)

customer_profit.insert(0, '仓中文名称', warehouse_name)

customer_profit['海外仓收入'] = 0
customer_profit['仓发配成本'] = 0
customer_profit['仓发配收入'] = 0

df_dict['成本明细集成'] = df_dict['成本明细集成'].rename(columns={'成本_客户编码':'客户编码'})

## 2 Filtering Data

In [ ]:
# Filter warehouse income
warehouse_income = df_dict['收入明细'][
    # (df_dict['收入明细']['签约二级'] == '美洲区') &
    (df_dict['收入明细']['产品标名称'] == '国际物流-海外仓配-海外仓产品')
]

# Filter warehouse delivery income
warehouse_delivery_income = df_dict['收入明细'][
    # (df_dict['收入明细']['签约二级'] == '美洲区') &
    (df_dict['收入明细']['产品标名称'].isin([
        '国际物流-海外仓配-海外仓产品-仓发配送',
        '国际物流-海外仓配-海外仓产品-仓发运输'
    ]))
]

# Filter warehouse delivery cost
warehouse_delivery_cost = df_dict['成本明细集成'][
    # (df_dict['收入明细']['签约二级'] == '美洲区') &
    (df_dict['成本明细集成']['产品标名称'].isin([
        '国际物流-海外仓配-海外仓产品-仓发配送',
        '国际物流-海外仓配-海外仓产品-仓发运输'
    ]))
]

# Filter delivery income
delivery_income = df_dict['收入明细'][
    # (df_dict['收入明细']['签约二级'] == '美洲区') &
    (df_dict['收入明细']['产品标名称'].isin([
        '国际物流-海外运配-海外纯配',
        '国际物流-海外运配-海外纯配-本土配送'
    ]))
]

# Filter delivery cost
delivery_cost = df_dict['成本明细集成'][
    # (df_dict['成本明细集成']['签约二级'] == '美洲区') &
    (df_dict['成本明细集成']['产品标名称'] == '国际物流-海外运配-海外纯配')
]

# Filter transport income
transport_income = df_dict['收入明细'][
    # (df_dict['收入明细']['签约二级'] == '美洲区') &
    (df_dict['收入明细']['产品标名称'].isin([
        '国际物流-海外运配-海外运输-陆运',
        '国际物流-跨境出口-国际运输-海运',
        '国际物流-跨境进口-国际运输-海运',
        '国际物流-海外运配-海外运输-空运',
        '国际物流-跨境进口-国际运输-陆运',
        '国际物流-海外运配-海外运输-海运',
        '国际物流-跨境出口-国际运输-空运',
        '国际物流-跨境出口-出口专线',
        '国际物流-跨境进口-国际运输-空运',
        '国际物流-跨境出口-国际运输-门到门',
        '国际物流-跨境出口-国际快递'
    ]))
]

# Filter transport cost
transport_cost = df_dict['成本明细集成'][
    # (df_dict['成本明细集成']['签约二级'] == '美洲区') &
    (df_dict['成本明细集成']['产品标名称'].isin([
        '国际物流-海外运配-海外运输-陆运',
        '国际物流-跨境出口-国际运输-海运',
        '国际物流-跨境进口-国际运输-海运',
        '国际物流-海外运配-海外运输-空运',
        '国际物流-跨境进口-国际运输-陆运',
        '国际物流-海外运配-海外运输-海运',
        '国际物流-跨境出口-国际运输-空运',
        '国际物流-跨境出口-出口专线',
        '国际物流-跨境进口-国际运输-空运',
        '国际物流-跨境出口-国际运输-门到门',
        '国际物流-跨境出口-国际快递',
        '专线小包'
    ]))
]

# Filter cooperator income
cooperator_income = df_dict['收入明细'][
    # (df_dict['收入明细']['签约二级'] == '美洲区') &
    (df_dict['收入明细']['仓类型'] == '协同') &
    (df_dict['收入明细']['产品标名称'].isin([
        '国际物流-海外仓配-海外仓产品',
        '国际物流-海外仓配-海外仓产品-仓发配送',
        '国际物流-海外仓配-海外仓产品-仓发运输'
    ]))
]

# Filter cooperator cost
cooperator_cost = df_dict['成本明细集成'][
    # (df_dict['成本明细集成']['签约二级'] == '美洲区') &
    (df_dict['成本明细集成']['仓类型'] == '协同') &
    (df_dict['成本明细集成']['产品标名称'].isin([
        '国际物流-海外仓配-海外仓产品',
        '国际物流-海外仓配-海外仓产品-仓发配送',
        '国际物流-海外仓配-海外仓产品-仓发运输'
    ]))
]

## 3 Functions to Add Cost and Revenue Data to Warehouse Report

In [ ]:
def add_cost_income(df, customer_profit, col):
    for index, row in df.iterrows():
        matched_rows = customer_profit[(customer_profit['KH'] == row['客户编码']) & (customer_profit['仓中文名称'] == row['仓名称'])]

        if not matched_rows.empty:
            customer_profit.loc[matched_rows.index, col] = matched_rows[col].sum() + row['不含税CNY']

        elif not customer_profit[customer_profit['仓中文名称'] == row['仓名称']].empty:
            new_row = pd.DataFrame({
                'KH': [row['客户编码']],
                '仓中文名称': [row['仓名称']],
                col: [row['不含税CNY']]
            })
            customer_profit = pd.concat([customer_profit, new_row], ignore_index=True)

    return customer_profit

def update_all_customer(df, all_customer, col):
    for index, row in df.iterrows():
        matched_rows = all_customer[all_customer['KH'] == row['客户编码']]

        if not matched_rows.empty:
            all_customer.loc[matched_rows.index, col] = matched_rows[col].sum() + row['不含税CNY']
        else:
            new_row = pd.DataFrame({
                'KH': [row['客户编码']],
                col: [row['不含税CNY']]
            })
            all_customer = pd.concat([all_customer, new_row], ignore_index=True)

    return all_customer

In [ ]:
customer_profit = add_cost_income(warehouse_income, customer_profit, '海外仓收入')
customer_profit = add_cost_income(warehouse_delivery_cost, customer_profit, '仓发配成本')
customer_profit = add_cost_income(warehouse_delivery_income, customer_profit, '仓发配收入')

## 4 Calculate Profit and Loss

In [ ]:
customer_profit = customer_profit.fillna(0)
customer_profit['海外仓损益'] = customer_profit['海外仓收入'] - customer_profit['海外仓成本']
customer_profit['仓发配损益'] = customer_profit['仓发配收入'] - customer_profit['仓发配成本']
customer_profit['仓配收入'] = customer_profit['海外仓收入'] + customer_profit['仓发配收入']
customer_profit['仓配成本'] = customer_profit['海外仓成本'] + customer_profit['仓发配成本']
customer_profit['仓配整体损益'] = customer_profit['仓配收入'] - customer_profit['仓配成本']

## 5 Generate a Customer P&L Statement for All Products

In [ ]:
all_customer = customer_profit.groupby('KH').agg({
    '海外仓损益': 'sum',
    '仓发配损益': 'sum',
    '仓配收入': 'sum',
    '仓配成本': 'sum',
    '仓配整体损益': 'sum'
}).reset_index()

all_customer['纯配成本'] = 0
all_customer['纯配收入'] = 0
all_customer['运输成本'] = 0
all_customer['运输收入'] = 0
all_customer['协同仓成本'] = 0
all_customer['协同仓收入'] = 0

all_customer = update_all_customer(delivery_cost, all_customer, '纯配成本')
all_customer = update_all_customer(delivery_income, all_customer, '纯配收入')
all_customer = update_all_customer(transport_cost, all_customer, '运输成本')
all_customer = update_all_customer(transport_income, all_customer, '运输收入')
all_customer = update_all_customer(cooperator_cost, all_customer, '协同仓成本')
all_customer = update_all_customer(cooperator_income, all_customer, '协同仓收入')

all_customer = all_customer.fillna(0)

kh_0_summary = all_customer[all_customer['KH'] == 0].sum(numeric_only=True)
kh_0_summary['KH'] = 0
all_customer = all_customer[all_customer['KH'] != 0]
all_customer = pd.concat([all_customer, pd.DataFrame([kh_0_summary])], ignore_index=True)

all_customer['纯配损益'] = all_customer['纯配收入'] - all_customer['纯配成本']
all_customer['运输损益'] = all_customer['运输收入'] - all_customer['运输成本']
all_customer['协同仓损益'] = all_customer['协同仓收入'] - all_customer['协同仓成本']

all_customer['收入总计'] = all_customer['纯配收入'] + all_customer['运输收入'] + all_customer['协同仓收入'] + all_customer['仓配收入']
all_customer['成本总计'] = all_customer['纯配成本'] + all_customer['运输成本'] + all_customer['协同仓成本'] + all_customer['仓配成本']
all_customer['总体损益'] = all_customer['收入总计'] - all_customer['成本总计']
all_customer['毛利率'] = all_customer['总体损益']/all_customer['收入总计']

all_customer.loc[(all_customer['纯配收入'] != 0) & (all_customer['纯配成本'] == 0), '纯配成本'] = '成本缺失'
all_customer.loc[(all_customer['纯配成本'] != 0) & (all_customer['纯配收入'] == 0), '纯配收入'] = '收入缺失'
all_customer.loc[(all_customer['运输收入'] != 0) & (all_customer['运输成本'] == 0), '运输成本'] = '成本缺失'
all_customer.loc[(all_customer['运输成本'] != 0) & (all_customer['运输收入'] == 0), '运输收入'] = '收入缺失'
all_customer.loc[(all_customer['协同仓收入'] != 0) & (all_customer['协同仓成本'] == 0), '协同仓成本'] = '成本缺失'
all_customer.loc[(all_customer['协同仓成本'] != 0) & (all_customer['协同仓收入'] == 0), '协同仓收入'] = '收入缺失'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    all_customer.to_excel(writer, sheet_name='含纯配运输', index=False)

## 6 Generate a Customer in Warehouse P&L Statement

In [ ]:
columns_to_convert = ['海外仓成本', '海外仓收入', '仓发配成本', '仓发配收入', '海外仓损益', '仓发配损益', '仓配收入', '仓配成本', '仓配整体损益']
customer_profit[columns_to_convert] = customer_profit[columns_to_convert].apply(pd.to_numeric, errors='coerce').fillna(0)

subtotals = customer_profit.groupby('仓中文名称')[columns_to_convert].sum()

subtotals['仓中文名称'] = subtotals.index
subtotals['客户名称'] = "Subtotal"

subtotal_df = pd.concat([customer_profit, subtotals], ignore_index=True)
subtotal_df.sort_values(by=['仓中文名称', 'KH'], inplace=True)

total_row = subtotal_df[subtotal_df['客户名称'] == 'Subtotal'].sum(numeric_only=True)
total_row['仓中文名称'] = 'Total'
total_dict = total_row.to_dict()
customer_profit = pd.concat([subtotal_df, pd.DataFrame([total_dict])], ignore_index=True)

In [ ]:
customer_profit.loc[(customer_profit['海外仓收入'] != 0) & (customer_profit['海外仓成本'] == 0), '海外仓成本'] = '成本缺失'
customer_profit.loc[(customer_profit['海外仓成本'] != 0) & (customer_profit['海外仓收入'] == 0), '海外仓收入'] = '收入缺失'

customer_profit.loc[(customer_profit['仓发配收入'] != 0) & (customer_profit['仓发配成本'] == 0), '仓发配成本'] = '成本缺失'
customer_profit.loc[(customer_profit['仓发配成本'] != 0) & (customer_profit['仓发配收入'] == 0), '仓发配收入'] = '收入缺失'

In [ ]:
with pd.ExcelWriter(output_file, engine='openpyxl', mode='a') as writer:
    customer_profit.to_excel(writer, sheet_name='自营仓整体', index=False)